# Demo3：自分で学習モデルから作るデモ

## ZIPファイルをアップロード

In [ ]:
# @markdown 実行すると`ファイル選択`ボタンが表示されますので、クリックしてZIPファイルをアップロードしてください

from google.colab import files

# ファイルアップロードのダイアログを表示
uploaded = files.upload()

# アップロードされたファイル名を表示
for name in uploaded.keys():
  print(f'アップロード完了: {name}')

## ZIPファイルから学習データセット作成

In [ ]:
# @markdown 学習データセットを作成します

import zipfile
import os
import json
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

VISUALIZE = False  # 一部のデータの可視化


# 1. Zipファイルの展開
extraction_path = '/content/'
os.makedirs(extraction_path, exist_ok=True)

# /content/ディレクトリ内のzipファイルを検索し、最新のものを選択
zip_files = glob.glob(os.path.join(extraction_path, '*.zip'))

if not zip_files:
    print(f'エラー: "{extraction_path}" にzipファイルが見つかりません。')
    exit()

# 最新のzipファイルを見つける (最終更新時間でソート)
zip_file_path = max(zip_files, key=os.path.getmtime)

print(f'最新のzipファイル: "{zip_file_path}" を処理対象とします。')

try:
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extraction_path)
    print(f'Zipファイル "{zip_file_path}" が "{extraction_path}" に正常に展開されました。')
except FileNotFoundError:
    print(f'エラー: 指定されたzipファイル "{zip_file_path}" が見つかりません。')
except zipfile.BadZipFile:
    print(f'エラー: "{zip_file_path}" は有効なzipファイルではありません。ファイルが破損しているか、zip形式ではありません。')
except Exception as e:
    print(f'zipファイルの展開中にエラーが発生しました: {e}')

# 2. メタデータの読み込み
meta_json_path = '/content/meta.json'
try:
    with open(meta_json_path, 'r') as f:
        meta_data = json.load(f)
    print('meta.json が正常に読み込まれました。')
except Exception as e:
    print(f'エラー: meta.json の読み込み中に問題が発生しました: {e}')
    exit()

file_labels = {}
label_names = meta_data['labelNames']

for segment in meta_data['segments']:
    file_name = segment['file']
    label = segment['label']
    file_labels[file_name] = label
print(f'抽出されたファイルごとのラベル数: {len(file_labels)}')
print(f'抽出されたラベル名: {label_names}')

# 3. CSVファイルの処理とデータ抽出
csv_directory = extraction_path
csv_files = glob.glob(os.path.join(csv_directory, 'seg_*.csv'))

X_data = []
y_labels = []

for file_path in csv_files:
    try:
        df = pd.read_csv(file_path)
        X_data.append(df.values)
        base_file_name = os.path.basename(file_path)
        label = file_labels[base_file_name]
        y_labels.append(label)
    except Exception as e:
        print(f"エラー: {file_path} の処理中に問題が発生しました: {e}")

print(f"処理されたCSVファイルの数: {len(X_data)}")
if X_data:
    print(f"最初のデータサンプルの形状: {X_data[0].shape}")
else:
    print("処理されたデータがありません。")

# 4. 学習データの整形とNumPy配列への変換
max_sequence_len = 0
for arr in X_data:
    if arr.shape[0] > max_sequence_len:
        max_sequence_len = arr.shape[0]

padded_X_data = []
for arr in X_data:
    padding_needed = max_sequence_len - arr.shape[0]
    if padding_needed > 0:
        padded_arr = np.pad(arr, ((0, padding_needed), (0, 0)), 'constant', constant_values=0)
        padded_X_data.append(padded_arr)
    else:
        padded_X_data.append(arr)

X = np.array(padded_X_data)
y = np.array(y_labels)

print(f"Xの形状: {X.shape}")
print(f"Xのデータ型: {X.dtype}")
print(f"yの形状: {y.shape}")
print(f"yのデータ型: {y.dtype}")

# 5. サンプル波形の可視化（日本語対応）
if VISUALIZE:

  # 日本語フォントのインストールと設定
  !apt-get -y install fonts-ipafont-gothic > /dev/null 2>&1 # 出力を抑制
  font_path = '/usr/share/fonts/opentype/ipafont-gothic/ipag.ttf'
  fm.fontManager.addfont(font_path)
  prop = fm.FontProperties(fname=font_path)
  actual_font_name = prop.get_name()
  plt.rcParams['font.family'] = actual_font_name

  num_samples_to_visualize = 3 # 最初の3つのサンプルを可視化

  for i in range(min(num_samples_to_visualize, X.shape[0])):
      sample_data = X[i]
      sample_label_index = y[i]
      sample_label_name = label_names[str(sample_label_index)]

      plt.figure(figsize=(12, 6))
      plt.plot(sample_data[:, 0], label='Feature 0 (x)')
      plt.plot(sample_data[:, 1], label='Feature 1 (y)')
      plt.plot(sample_data[:, 2], label='Feature 2 (z)')

      plt.title(f'サンプル波形 - サンプル {i}, ラベル: {sample_label_name} (インデックス: {sample_label_index})')
      plt.xlabel('タイムステップ')
      plt.ylabel('振幅')
      plt.legend()
      plt.grid(True)
      plt.tight_layout()
      plt.show()

print('\n全ての処理が完了しました。次のステップに進む準備ができました')

## micro:bit軽量版のコードで学習

In [ ]:
# @markdown micro:bit用リの推論モデルを作成します

import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split


# ============================================================
# trainとtestの分割
# ============================================================
TRAIN_RATIO = 1.0   # 100:0 データが少ないことを見込んで、すべてのデータをtrainデータとtrainデータに割り当てるパターン
# TRAIN_RATIO = 0.9   # 90:10
# TRAIN_RATIO = 0.8   # 80:20
# TRAIN_RATIO = 0.7   # 70:30

if TRAIN_RATIO >= 1.0:
    X_train, X_test, y_train, y_test = X, X, y, y
else:
    X_train, X_test, y_train, y_test = train_test_split(
        X,
        y,
        train_size=TRAIN_RATIO,
        shuffle=True,
        stratify=y,      # クラス比を保つ
        random_state=42
    )


# ============================================================
# 設定（micro:bit向け）
# ============================================================
N_RESERVOIR     = 64
SPECTRAL_RADIUS = 0.9
LEAK_RATE       = 0.5
DENSITY         = 0.1
INPUT_SCALING   = 0.8
RIDGE_REG       = 1e-5
SEED            = 123
WASHOUT         = 10        # 完全オンラインの安定化（学習で先頭を捨てる）


# ============================================================
# 標準化（micro:bit側 MEAN/SCALE にコピーする）
# ============================================================
def compute_mean_scale(X):
    # X: (N,T,K)
    flat = X.reshape(-1, X.shape[2]).astype(np.float32)
    mean = flat.mean(axis=0)
    std  = flat.std(axis=0)
    std[std == 0] = 1.0
    return mean.astype(np.float32), std.astype(np.float32)

def standardize(X, mean, scale):
    return ((X.astype(np.float32) - mean) / scale).astype(np.float32)


# ============================================================
# ESN学習（全フレーム教師あり Ridge / states保存なし）
#    micro:bit形式: pre = W*x + Win*u + B_res
#    PC側は Win_full = [bias, inputs] を持ち、bias列は B_resへ吸収して出す
# ============================================================
class ESNTrainer:
    def __init__(self, n_inputs,
                 n_reservoir=N_RESERVOIR,
                 spectral_radius=SPECTRAL_RADIUS,
                 input_scaling=INPUT_SCALING,
                 leak_rate=LEAK_RATE,
                 ridge_reg=RIDGE_REG,
                 density=DENSITY,
                 seed=SEED):

        self.K = int(n_inputs)
        self.N = int(n_reservoir)
        self.sr_target = float(spectral_radius)
        self.in_scale = float(input_scaling)
        self.leak = float(leak_rate)
        self.ridge = float(ridge_reg)
        self.density = float(density)

        rng = np.random.default_rng(seed)

        # Win_full: (N, 1+K) 先頭がバイアス
        self.W_in_full = (rng.uniform(-1, 1, (self.N, self.K + 1)) * self.in_scale).astype(np.float32)

        # Reservoir W: (N,N) sparse
        W = rng.uniform(-1, 1, (self.N, self.N)).astype(np.float32)
        mask = (rng.uniform(0, 1, (self.N, self.N)) < self.density).astype(np.float32)
        W *= mask

        # spectral radius normalize（PCだけ）
        eigvals = np.linalg.eigvals(W.astype(np.float64))
        sr = float(np.max(np.abs(eigvals)))
        if sr == 0:
            raise ValueError("spectral radius is zero. Increase density.")
        self.W = (W * (self.sr_target / sr)).astype(np.float32)

        # W_out: (N+1, M)  ※出力側バイアス込み（最後の+1）
        self.W_out = None

    def forward_states_stream(self, u_seq):
        """u_seq: (T,K) → 各時刻の x(t) を順に返す（保存しない）"""
        x = np.zeros(self.N, dtype=np.float32)
        for t in range(u_seq.shape[0]):
            u = u_seq[t]
            in_vec = np.concatenate([np.array([1.0], np.float32), u])  # (1+K,)
            pre = self.W_in_full @ in_vec + self.W @ x
            x_new = np.tanh(pre).astype(np.float32)
            if self.leak >= 0.999999:
                x = x_new
            else:
                x = (1.0 - self.leak) * x + self.leak * x_new
            yield x

    def fit_framewise_ridge_streaming(self, X, y, num_classes, washout=WASHOUT):
        """
        完全オンライン向け：毎時刻の状態 x(t) を教師ありで使う
        X: (Nseq,T,K)（標準化済み推奨）
        y: (Nseq,)（シーケンスラベル：全フレーム同ラベル）
        """
        Nseq, T, K = X.shape
        M = int(num_classes)

        dim = self.N + 1  # r=[x,1]
        S = np.zeros((dim, dim), dtype=np.float64)
        U = np.zeros((dim, M), dtype=np.float64)

        for i in range(Nseq):
            yi = int(y[i])
            y_one = np.zeros(M, dtype=np.float64)
            y_one[yi] = 1.0

            t = 0
            for x in self.forward_states_stream(X[i]):
                if t >= washout:
                    r = np.concatenate([x, np.array([1.0], np.float32)]).astype(np.float64)
                    S += np.outer(r, r)
                    U += np.outer(r, y_one)
                t += 1

        A = S + self.ridge * np.eye(dim, dtype=np.float64)
        self.W_out = np.linalg.solve(A, U).astype(np.float32)
        return self


# ============================================================
# 量子化（int8）＆ CSR化（Wのみ）
# ============================================================
def quantize_int8(mat, clip=127):
    max_abs = float(np.max(np.abs(mat))) if mat.size else 0.0
    if max_abs == 0.0:
        return np.zeros_like(mat, dtype=np.int32), 1.0
    scale = max_abs / clip
    q = np.round(mat / scale).astype(np.int32)
    q = np.clip(q, -clip, clip)
    return q, scale

def dense_q_to_csr(q_mat):
    N = q_mat.shape[0]
    row_ptr = [0]
    col_idx = []
    val_q = []
    nnz = 0
    for i in range(N):
        row = q_mat[i]
        for j, v in enumerate(row):
            if v != 0:
                col_idx.append(int(j))
                val_q.append(int(v))
                nnz += 1
        row_ptr.append(nnz)
    return row_ptr, col_idx, val_q


# ============================================================
# 完全オンラインをPCで再現（量子化モデルで）
#    → 各フレームの「線形スコア」を生成（平均評価に使用）
# ============================================================
def simulate_timewise_scores_quantized(X_seq_std, consts):
    """
    X_seq_std: (T,K) float（標準化済み）
    consts: dict
    戻り：scores_t (T, M) 各時刻の線形スコア（softmax前）
    """
    N = consts["N"]
    K = consts["K"]
    M = consts["M"]

    row_ptr = consts["RES_ROW_PTR"]
    col_idx = consts["RES_COL_IDX"]
    val_q  = consts["RES_VAL_Q"]
    res_scale = consts["RES_SCALE"]

    in_val_q = consts["IN_VAL_Q"]
    in_scale = consts["IN_SCALE"]

    W_OUT_Q = consts["W_OUT_Q"]          # (M, N+1) int
    W_OUT_SCALE = consts["W_OUT_SCALE"]  # float
    B_RES_Q = consts["B_RES_Q"]          # (N,) int
    B_RES_SCALE = consts["B_RES_SCALE"]  # float

    LEAK = consts["LEAK"]

    x = [0.0] * N
    scores_t = []

    for u in X_seq_std:
        # --- state update ---
        pre = [0.0] * N
        for i in range(N):
            s = 0.0

            # W*x (CSR)
            for p in range(row_ptr[i], row_ptr[i + 1]):
                j = col_idx[p]
                s += (val_q[p] * res_scale) * x[j]

            # Win*u (dense int8)
            base = i * K
            for kk in range(K):
                s += (in_val_q[base + kk] * in_scale) * float(u[kk])

            # bias (int8 -> float)
            s += (B_RES_Q[i] * B_RES_SCALE)

            pre[i] = s

        if LEAK >= 0.999999:
            for i in range(N):
                x[i] = float(np.tanh(pre[i]))
        else:
            for i in range(N):
                xn = float(np.tanh(pre[i]))
                x[i] = (1.0 - LEAK) * x[i] + LEAK * xn

        # --- output (each frame) : 線形スコア ---
        scores = [0.0] * M
        for m in range(M):
            row_q = W_OUT_Q[m]  # length N+1
            s = 0.0

            for i in range(N):
                s += (row_q[i] * W_OUT_SCALE) * x[i]

            # bias
            s += (row_q[N] * W_OUT_SCALE)

            scores[m] = s

        scores_t.append(scores)

    return np.array(scores_t, dtype=np.float32)

def predict_sequence_by_mean_linear_score(scores_t):
    """
    scores_t: (T, M) 各時刻の線形スコア
    → 平均して argmax を返す
    """
    if len(scores_t) == 0:
        return 0

    mean_scores = scores_t.mean(axis=0)  # (M,)
    return int(np.argmax(mean_scores))


# ============================================================
# 実行：学習→量子化→完全オンライン評価→定数出力
# ============================================================
# ラベルが 0..M-1 でない可能性に備え、再マッピング
classes = sorted(list(set(int(v) for v in np.unique(y_train))))
label_to_idx = {c:i for i,c in enumerate(classes)}
y_train_idx = np.array([label_to_idx[int(v)] for v in y_train], dtype=np.int32)
y_test_idx  = np.array([label_to_idx[int(v)] for v in y_test],  dtype=np.int32)

K = X_train.shape[2]
M = len(classes)

# 標準化
mean, scale = compute_mean_scale(X_train)
Xtr = standardize(X_train, mean, scale)
Xte = standardize(X_test,  mean, scale)

# 学習
esn = ESNTrainer(
    n_inputs=K,
    n_reservoir=N_RESERVOIR,
    spectral_radius=SPECTRAL_RADIUS,
    input_scaling=INPUT_SCALING,
    leak_rate=LEAK_RATE,
    ridge_reg=RIDGE_REG,
    density=DENSITY,
    seed=SEED
).fit_framewise_ridge_streaming(Xtr, y_train_idx, num_classes=M, washout=WASHOUT)

print("Trained. W_out shape:", esn.W_out.shape)


# ============================================================
# 量子化＆CSR化（micro:bit形式）
# ============================================================
# W: CSR + int8
qW, RES_SCALE = quantize_int8(esn.W)
RES_ROW_PTR, RES_COL_IDX, RES_VAL_Q = dense_q_to_csr(qW)

# Win: bias列はB_RESへ、残りは int8 でフラット化
B_RES_F = esn.W_in_full[:, 0].astype(np.float32)       # float (N,)
qIn, IN_SCALE = quantize_int8(esn.W_in_full[:, 1:])    # int8相当 (N,K)
IN_VAL_Q = qIn.reshape(-1).astype(int).tolist()

# B_RES を int8 + scale に量子化
B_RES_Q, B_RES_SCALE = quantize_int8(B_RES_F)

# W_OUT を int8 + scale に量子化（microbit側は (M, N+1)）
W_OUT_F = esn.W_out.T.astype(np.float32)  # (M, N+1) float
W_OUT_Q, W_OUT_SCALE = quantize_int8(W_OUT_F)

# 定数辞書（micro:bitへ出す形）
consts = {
    "N": N_RESERVOIR, "K": K, "M": M,
    "RES_ROW_PTR": RES_ROW_PTR,
    "RES_COL_IDX": RES_COL_IDX,
    "RES_VAL_Q": RES_VAL_Q,
    "RES_SCALE": float(RES_SCALE),
    "IN_VAL_Q": IN_VAL_Q,
    "IN_SCALE": float(IN_SCALE),

    "W_OUT_Q": W_OUT_Q.astype(int).tolist(),     # (M, N+1)
    "W_OUT_SCALE": float(W_OUT_SCALE),
    "B_RES_Q": B_RES_Q.astype(int).tolist(),     # (N,)
    "B_RES_SCALE": float(B_RES_SCALE),

    "LEAK": float(LEAK_RATE),
    "MEAN": [float(v) for v in mean.tolist()],
    "SCALE": [float(v) for v in scale.tolist()],
}


# ============================================================
# 完全オンライン評価（量子化モデルで線形スコアを生成）
# ============================================================
y_pred_mean = []
for i in range(Xte.shape[0]):
  scores_t = simulate_timewise_scores_quantized(Xte[i], consts)  # (T, M)
  y_pred_mean.append(
      predict_sequence_by_mean_linear_score(scores_t)
  )

y_pred_mean = np.array(y_pred_mean, dtype=np.int32)

print("Test accuracy (ONLINE, mean linear score):",
      accuracy_score(y_test_idx, y_pred_mean))
print("Confusion matrix:\n",
      confusion_matrix(y_test_idx, y_pred_mean))


# ============================================================
# 追加：1フレームごとの評価（frame-wise accuracy）
#    ・各フレームでの瞬間的な判定性能を見る
#    ・シーケンス平均や最終フレームは使わない
# ============================================================

all_pred_frames = []
all_true_frames = []

for i in range(Xte.shape[0]):
    # 各フレームの線形スコアを計算 (T, M)
    scores_t = simulate_timewise_scores_quantized(Xte[i], consts)

    # 各フレームで argmax を取り、クラスID列 (T,) を作る
    pred_t = np.argmax(scores_t, axis=1).astype(np.int32)

    all_pred_frames.append(pred_t)

    # 正解ラベルはシーケンス単位なので、全フレーム同一ラベル
    true_t = np.full(pred_t.shape[0], y_test_idx[i], dtype=np.int32)
    all_true_frames.append(true_t)

# 全シーケンス分を連結してフレーム単位で評価
all_pred_frames = np.concatenate(all_pred_frames)
all_true_frames = np.concatenate(all_true_frames)

print("Frame-wise Test accuracy:",
      accuracy_score(all_true_frames, all_pred_frames))
print("Frame-wise Confusion matrix:\n",
      confusion_matrix(all_true_frames, all_pred_frames))

## モデルを埋め込んだmicro:bit用推論コードをダウンロード

In [ ]:
# @markdown 実行するとmain.pyがダウンロードされます

from google.colab import files

code_string = f"""
# main.py — micro:bit (MicroPython) ESN リアルタイム推論（完全オンライン / CSR + int8）
import microbit
import math

MG2G = 1.0 / 1000.0

# ----------------------------
# tanh（MicroPythonにmath.tanhが無い場合があるため）
# ----------------------------
def _tanh(z):
    if z > 8.0:
        return 1.0
    if z < -8.0:
        return -1.0
    e = math.exp(2.0 * z)
    return (e - 1.0) / (e + 1.0)

# =========================================================
# ここにPC側で出力したモデル定数を貼り付け
# N,K,M, RES_ROW_PTR, RES_COL_IDX, RES_VAL_Q, RES_SCALE,
# IN_VAL_Q, IN_SCALE,
# W_OUT_Q, W_OUT_SCALE,
# B_RES_Q, B_RES_SCALE,
# LEAK, ADD_BIAS, MEAN, SCALE, CLASSES
# =========================================================
N = {consts['N']}
K = {consts['K']}
M = {consts['M']}
RES_ROW_PTR = {consts['RES_ROW_PTR']}
RES_COL_IDX = {consts['RES_COL_IDX']}
RES_VAL_Q = {consts['RES_VAL_Q']}
RES_SCALE = {consts['RES_SCALE']}
IN_VAL_Q = {consts['IN_VAL_Q']}
IN_SCALE = {consts['IN_SCALE']}
W_OUT_Q = {consts['W_OUT_Q']}
W_OUT_SCALE = {consts['W_OUT_SCALE']}
B_RES_Q = {consts['B_RES_Q']}
B_RES_SCALE = {consts['B_RES_SCALE']}
LEAK = {consts['LEAK']}
ADD_BIAS = True
MEAN = {consts['MEAN']}
SCALE = {consts['SCALE']}
CLASSES = {classes}
# =========================================================

# ----------------------------
# 推論設定（完全オンライン用）
# ----------------------------
SAMPLE_MS = 10
WARMUP_FRAMES = 20
EMA_BETA = 0.20
DEBOUNCE = 3
MARGIN = 0.0

# ----------------------------
# 標準化（PC側 MEAN/SCALE と一致させる）
# ----------------------------
def standardize_vec(u):
    if MEAN is None or SCALE is None:
        return u[:]
    out = [0.0] * K
    for i in range(K):
        s = SCALE[i] if SCALE[i] != 0 else 1.0
        out[i] = (u[i] - MEAN[i]) / s
    return out

# ----------------------------
# ESN 1 step（CSR + int8）
# x: (N,) state
# u: (K,) standardized input
# ----------------------------
def esn_step_inplace(x, u):
    pre = [0.0] * N
    for i in range(N):
        s = 0.0

        # Reservoir W*x (CSR)
        start = RES_ROW_PTR[i]
        end   = RES_ROW_PTR[i + 1]
        for p in range(start, end):
            j = RES_COL_IDX[p]
            s += (RES_VAL_Q[p] * RES_SCALE) * x[j]

        # Input Win*u (dense int8, row-major flat)
        base = i * K
        for kk in range(K):
            s += (IN_VAL_Q[base + kk] * IN_SCALE) * u[kk]

        # Bias (int8 + scale)
        s += (B_RES_Q[i] * B_RES_SCALE)

        pre[i] = s

    # tanh + leak
    if LEAK == 1.0:
        for i in range(N):
            x[i] = _tanh(pre[i])
    else:
        for i in range(N):
            xn = _tanh(pre[i])
            x[i] = (1.0 - LEAK) * x[i] + LEAK * xn

# ----------------------------
# 1フレーム推論（完全オンライン）
# scores: (M,)
# ----------------------------
def linear_scores_from_state(x):
    scores = [0.0] * M
    for m in range(M):
        row_q = W_OUT_Q[m]  # length N (+bias at N)
        s = 0.0
        for i in range(N):
            s += (row_q[i] * W_OUT_SCALE) * x[i]
        if ADD_BIAS:
            s += (row_q[N] * W_OUT_SCALE)
        scores[m] = s
    return scores

def argmax_and_margin(scores):
    best = 0
    bestv = scores[0]
    secondv = -1e30
    for i in range(M):
        v = scores[i]
        if v > bestv:
            secondv = bestv
            bestv = v
            best = i
        elif v > secondv:
            secondv = v
    return best, (bestv - secondv)

# ----------------------------
# 表示
# ----------------------------
def show_label(lbl):
    try:
        n = int(lbl)
    except:
        n = 0
    if n == 0:
        microbit.display.show('0')
    elif n == 1:
        microbit.display.show('1')
    elif n == 2:
        microbit.display.show('2')
    else:
        microbit.display.show('?')

# =========================================================
# main loop（完全オンライン）
# =========================================================
x_state = [0.0] * N

# EMAの状態
z_ema = None

# debounce状態
last_shown = None
cand = None
cand_count = 0

microbit.display.show('?')

frame = 0
while True:
    ax = microbit.accelerometer.get_x() * MG2G
    ay = microbit.accelerometer.get_y() * MG2G
    az = microbit.accelerometer.get_z() * MG2G

    u_std = standardize_vec([ax, ay, az])

    # state update
    esn_step_inplace(x_state, u_std)

    # warmup中は表示しない
    frame += 1
    if frame < WARMUP_FRAMES:
        microbit.sleep(SAMPLE_MS)
        continue

    # scores
    z = linear_scores_from_state(x_state)

    # EMA
    if z_ema is None:
        z_ema = z[:]
    else:
        b = EMA_BETA
        for i in range(M):
            z_ema[i] = (1.0 - b) * z_ema[i] + b * z[i]

    # predict with margin
    pred, gap = argmax_and_margin(z_ema)

    # マージンが小さい場合は切替抑制（last_shown維持）
    if MARGIN > 0.0 and last_shown is not None and gap < MARGIN:
        pred = last_shown

    # debounce
    if cand is None:
        cand = pred
        cand_count = 1
    else:
        if pred == cand:
            cand_count += 1
        else:
            cand = pred
            cand_count = 1

    if cand_count >= DEBOUNCE:
        # class index -> label
        try:
            out_label = CLASSES[cand]
        except:
            out_label = cand

        if out_label != last_shown:
            show_label(out_label)
            last_shown = out_label

    microbit.sleep(SAMPLE_MS)
""".strip()


# print(code_string)

# 1. code_string の内容を 'main.py' という名前で保存
with open('main.py', 'w', encoding='utf-8') as f:
    f.write(code_string)

# 2. そのファイルをローカルPCにダウンロード
files.download('main.py')

## おまけ：🎶リアルタイム推論に応じてメロディー出力


In [ ]:
# @markdown 実行するとmelody.pyがダウンロードされます

from google.colab import files

code_string = f"""
# main.py — micro:bit (MicroPython) ESN リアルタイム推論（完全オンライン / CSR + int8）+ メロディー
import microbit
import math
import music

MG2G = 1.0 / 1000.0

# ----------------------------
# tanh（MicroPythonにmath.tanhが無い場合があるため）
# ----------------------------
def _tanh(z):
    if z > 8.0:
        return 1.0
    if z < -8.0:
        return -1.0
    e = math.exp(2.0 * z)
    return (e - 1.0) / (e + 1.0)

# =========================================================
# ここにPC側で出力したモデル定数を貼り付け
# N,K,M, RES_ROW_PTR, RES_COL_IDX, RES_VAL_Q, RES_SCALE,
# IN_VAL_Q, IN_SCALE,
# W_OUT_Q, W_OUT_SCALE,
# B_RES_Q, B_RES_SCALE,
# LEAK, ADD_BIAS, MEAN, SCALE, CLASSES
# =========================================================
N = {consts['N']}
K = {consts['K']}
M = {consts['M']}
RES_ROW_PTR = {consts['RES_ROW_PTR']}
RES_COL_IDX = {consts['RES_COL_IDX']}
RES_VAL_Q = {consts['RES_VAL_Q']}
RES_SCALE = {consts['RES_SCALE']}
IN_VAL_Q = {consts['IN_VAL_Q']}
IN_SCALE = {consts['IN_SCALE']}
W_OUT_Q = {consts['W_OUT_Q']}
W_OUT_SCALE = {consts['W_OUT_SCALE']}
B_RES_Q = {consts['B_RES_Q']}
B_RES_SCALE = {consts['B_RES_SCALE']}
LEAK = {consts['LEAK']}
ADD_BIAS = True
MEAN = {consts['MEAN']}
SCALE = {consts['SCALE']}
CLASSES = {classes}
# =========================================================

# ----------------------------
# 推論設定（完全オンライン用）
# ----------------------------
SAMPLE_MS = 10
WARMUP_FRAMES = 20
EMA_BETA = 0.20
DEBOUNCE = 3
MARGIN = 0.0

# ----------------------------
# 標準化（PC側 MEAN/SCALE と一致させる）
# ----------------------------
def standardize_vec(u):
    if MEAN is None or SCALE is None:
        return u[:]
    out = [0.0] * K
    for i in range(K):
        s = SCALE[i] if SCALE[i] != 0 else 1.0
        out[i] = (u[i] - MEAN[i]) / s
    return out

# ----------------------------
# ESN 1 step（CSR + int8）
# x: (N,) state
# u: (K,) standardized input
# ----------------------------
def esn_step_inplace(x, u):
    pre = [0.0] * N
    for i in range(N):
        s = 0.0

        # Reservoir W*x (CSR)
        start = RES_ROW_PTR[i]
        end   = RES_ROW_PTR[i + 1]
        for p in range(start, end):
            j = RES_COL_IDX[p]
            s += (RES_VAL_Q[p] * RES_SCALE) * x[j]

        # Input Win*u (dense int8, row-major flat)
        base = i * K
        for kk in range(K):
            s += (IN_VAL_Q[base + kk] * IN_SCALE) * u[kk]

        # Bias (int8 + scale)
        s += (B_RES_Q[i] * B_RES_SCALE)

        pre[i] = s

    # tanh + leak
    if LEAK == 1.0:
        for i in range(N):
            x[i] = _tanh(pre[i])
    else:
        for i in range(N):
            xn = _tanh(pre[i])
            x[i] = (1.0 - LEAK) * x[i] + LEAK * xn

# ----------------------------
# 1フレーム推論（完全オンライン）
# scores: (M,)
# ----------------------------
def linear_scores_from_state(x):
    scores = [0.0] * M
    for m in range(M):
        row_q = W_OUT_Q[m]  # length N (+bias at N)
        s = 0.0
        for i in range(N):
            s += (row_q[i] * W_OUT_SCALE) * x[i]
        if ADD_BIAS:
            s += (row_q[N] * W_OUT_SCALE)
        scores[m] = s
    return scores

def argmax_and_margin(scores):
    best = 0
    bestv = scores[0]
    secondv = -1e30
    for i in range(M):
        v = scores[i]
        if v > bestv:
            secondv = bestv
            bestv = v
            best = i
        elif v > secondv:
            secondv = v
    return best, (bestv - secondv)

# ----------------------------
# 表示
# ----------------------------
def show_label(lbl):
    try:
        n = int(lbl)
    except:
        n = 0
    if n == 0:
        microbit.display.show('0')
    elif n == 1:
        microbit.display.show('1')
    elif n == 2:
        microbit.display.show('2')
    else:
        microbit.display.show('?')

# =========================================================
# 短音刻みメロディ（ノンブロッキング）
#  - 0: 無音
#  - 1: 歓喜の歌
#  - 2: ハッピーバースデートゥーユー
# ※「本家のmusic.ODE/music.NYANをそのまま再生」だとブロックしがちなので、
#   pitch(wait=False)で刻む方式にしています。
# =========================================================
# (Hz, ms) で並べる。freq=0 は休符。
MELODY_ODE = [
    (330,150),(330,150),(349,150),(392,150), # ミミファソ
    (392,150),(349,150),(330,150),(294,150), # ソファミレ
    (262,150),(262,150),(294,150),(330,150), # ドドレミ
    (330,200),(294,200),(294,250),(0,120),   # ミレレ-(休)
    (330,150),(330,150),(349,150),(392,150), # ミミファソ
    (392,150),(349,150),(330,150),(294,150), # ソファミレ
    (262,150),(262,150),(294,150),(330,150), # ドドレミ
    (294,200),(262,200),(262,250),(0,120),   # レドド-(休)
]
MELODY_HAPPY = [
    # --- フレーズ 1: ソ(低) ソ(低) ラ(低) ソ(低) ド(中) シ(低) ---
    (392,150),(392,200),(440,300),(392,300),(523,300),(494,450),(0,120),

    # --- フレーズ 2: ソ(低) ソ(低) ラ(低) ソ(低) レ(中) ド(中) ---
    (392,150),(392,200),(440,300),(392,300),(587,300),(523,450),(0,120),

    # --- フレーズ 3: ソ(低) ソ(低) ソ(高) ミ(中) ド(中) シ(低) ラ(低) ---
    (392,150),(392,200),(784,300),(659,300),(523,300),(494,300),(440,450),(0,120),

    # --- フレーズ 4: ファ(中) ファ(中) ミ(中) ド(中) レ(中) ド(中) ---
    (698,150),(698,200),(659,300),(523,300),(587,300),(523,500),
]

_cur_melody = None
_mel_idx = 0
_next_note_ms = 0

def _set_melody_for_label(lbl):
    global _cur_melody, _mel_idx, _next_note_ms
    try:
        n = int(lbl)
    except:
        n = 0

    music.stop()  # 切替時は必ず停止

    if n == 1:
        _cur_melody = MELODY_ODE
        _mel_idx = 0
        _next_note_ms = 0
    elif n == 2:
        _cur_melody = MELODY_HAPPY
        _mel_idx = 0
        _next_note_ms = 0

    else:
        # 0（無音）
        _cur_melody = None
        _mel_idx = 0
        _next_note_ms = 0

def _music_tick():
    #毎ループ呼ぶ。時間が来たら次の短音を鳴らす（ノンブロッキング）
    global _mel_idx, _next_note_ms

    if _cur_melody is None:
        return

    now = microbit.running_time()
    if now < _next_note_ms:
        return

    freq, dur = _cur_melody[_mel_idx]
    if freq <= 0:
        music.stop()
    else:
        # wait=False で推論ループを止めない
        music.pitch(int(freq), int(dur), wait=False)

    _next_note_ms = now + int(dur)
    _mel_idx += 1
    if _mel_idx >= len(_cur_melody):
        _mel_idx = 0  # ループ再生（不要ならここで _cur_melody=None にして終了）


# =========================================================
# main loop（完全オンライン）
# =========================================================
x_state = [0.0] * N

# EMAの状態
z_ema = None

# debounce状態
last_shown = None
cand = None
cand_count = 0

microbit.display.show('?')

frame = 0
while True:
    ax = microbit.accelerometer.get_x() * MG2G
    ay = microbit.accelerometer.get_y() * MG2G
    az = microbit.accelerometer.get_z() * MG2G

    u_std = standardize_vec([ax, ay, az])

    # state update
    esn_step_inplace(x_state, u_std)

    # warmup中は表示しない
    frame += 1
    if frame < WARMUP_FRAMES:
        microbit.sleep(SAMPLE_MS)
        continue

    # scores
    z = linear_scores_from_state(x_state)

    # EMA
    if z_ema is None:
        z_ema = z[:]
    else:
        b = EMA_BETA
        for i in range(M):
            z_ema[i] = (1.0 - b) * z_ema[i] + b * z[i]

    # predict with margin
    pred, gap = argmax_and_margin(z_ema)

    # マージンが小さい場合は切替抑制（last_shown維持）
    if MARGIN > 0.0 and last_shown is not None and gap < MARGIN:
        pred = last_shown

    # debounce
    if cand is None:
        cand = pred
        cand_count = 1
    else:
        if pred == cand:
            cand_count += 1
        else:
            cand = pred
            cand_count = 1

    if cand_count >= DEBOUNCE:
        # class index -> label
        try:
            out_label = CLASSES[cand]
        except:
            out_label = cand

        if out_label != last_shown:
            show_label(out_label)
            _set_melody_for_label(out_label)  # 0/1/2で切替
            last_shown = out_label

    _music_tick()  # 毎ループ刻む
    microbit.sleep(SAMPLE_MS)
""".strip()


# print(code_string)

# 1. code_string の内容を 'main.py' という名前で保存
with open('melody.py', 'w', encoding='utf-8') as f:
    f.write(code_string)

# 2. そのファイルをローカルPCにダウンロード
files.download('melody.py')